# Library

In [2]:
import numpy as np
import pandas as pd

# Functions

In [9]:
def SITDL(tuples, distance, slot):
    
    groups = tuples.sort_values(['distance-m'], ascending=True, ignore_index=True).groupby(['center_poi'])
    centerDistance = distance/slot
    sentences = list()
    
    columns = np.arange(0, int(centerDistance+1))
    columns = [str(w) for w in columns]
    
    for group_name, df_group in groups:

        df_group = df_group.dropna()
        
        for i in np.arange (0, distance, slot):   
            if(i == slot):
                rows = df_group[(df_group['distance-m'].astype(float) > i) & (df_group['distance-m'].astype(float) <= (i+slot))]
            else:
                rows = df_group[(df_group['distance-m'].astype(float) >= i) & (df_group['distance-m'].astype(float) <= (i+slot))]
        
            for id, row in rows.iterrows():
                subsentence = ['s'] * int(centerDistance+1)
                subsentence[0] = row['center_poi']
                subsentence[int((i/slot) + 1)] = row['context_poi']

                sentences.append(subsentence)          
    return [sentences, columns]

In [28]:
def Tuple2Sentence(n, wgt, h):
    
    print('Converting tuple of bin:', n)
    distance = (n+1)*h

    layers = [1, 2, 3]

    for l in layers:

        try:

            file_name = 'file-p.parquet'

            tuples = pd.read_parquet(file_name)

            windows = [9, 11]

            for w in windows:

               
                slot = (distance/((w-1)/2))
                
                [sentences_window, columns] = SITDL(tuples, distance, slot)

                sentences_window_file_name = './Austin/w05/Sentences ITDL Partial Bins Single Layers/austin-sl-sentences-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-wnd'+str(w)+'-l'+str(l)+'.parquet'
        
                sentences_window = pd.DataFrame(sentences_window, columns = columns)
                sentences_window.columns = sentences_window.columns.astype(str)
                sentences_window = sentences_window.dropna()

                print("Saving file:", sentences_window_file_name) 
                sentences_window.to_parquet(sentences_window_file_name, compression='brotli', index = False)


        except Exception as e:
            print(str(e))

# Tuple2Sentence

Conversão em Paralelo

In [ ]:
for n in range(0, 27):
    Tuple2Sentence(n, 0.5, 100)
print('Process finished.')